In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
games = pd.read_csv('../data/games.csv')
games_details = pd.read_csv('../data/games_details.csv', low_memory=False)
players = pd.read_csv('../data/players.csv')
teams = pd.read_csv('../data/teams.csv')
ranking = pd.read_csv('../data/ranking.csv')

In [9]:
def check_missing_values(df, df_name):

    missing = df.isnull().sum()
    missing_percent = (missing / len(df)) * 100
    
    missing_df = pd.DataFrame({
        'Missing_Count': missing,
        'Percent': missing_percent
    })
    missing_df = missing_df[missing_df.Missing_Count > 0].sort_values('Percent', ascending=False)
    
    print(f"\nMissing values in {df_name}:")
    print(missing_df)
    return missing_df

check_missing_values(games, 'games')
check_missing_values(games_details, 'games_details')
check_missing_values(players, 'players')
check_missing_values(teams, 'teams')
check_missing_values(ranking, 'ranking')


Missing values in games:
              Missing_Count   Percent
PTS_home                 99  0.371468
FG_PCT_home              99  0.371468
FT_PCT_home              99  0.371468
FG3_PCT_home             99  0.371468
AST_home                 99  0.371468
REB_home                 99  0.371468
PTS_away                 99  0.371468
FG_PCT_away              99  0.371468
FT_PCT_away              99  0.371468
FG3_PCT_away             99  0.371468
AST_away                 99  0.371468
REB_away                 99  0.371468

Missing values in games_details:
                Missing_Count    Percent
NICKNAME               615591  92.067787
COMMENT                558939  83.594914
START_POSITION         412863  61.747788
PLUS_MINUS             133351  19.943975
MIN                    109690  16.405236
FGA                    109690  16.405236
FGM                    109690  16.405236
FG_PCT                 109690  16.405236
FG3M                   109690  16.405236
FG3_PCT                109690  16.40

,Missing_Count,Percent
RETURNTOPLAY,206352,98.103089


In [10]:
def clean_games_details(df):

    df = df.dropna(subset=['MIN'])
    df = df[df['MIN'] != '0:00']
    df = df[df['MIN'] != '0']
    
    essential_cols = ['PLAYER_NAME', 'TEAM_ID', 'GAME_ID', 'PLAYER_ID']
    df = df.dropna(subset=essential_cols)
    
    stat_columns = ['FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
                   'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS']
    
    for col in stat_columns:
        if col in df.columns:
            if col.endswith('_PCT'):
                # For percentage columns, use 0 or calculate if possible
                if col == 'FG_PCT' and 'FGM' in df.columns and 'FGA' in df.columns:
                    df[col] = df[col].fillna((df['FGM'] / df['FGA'].replace(0, 1)) * 100)
                elif col == 'FG3_PCT' and 'FG3M' in df.columns and 'FG3A' in df.columns:
                    df[col] = df[col].fillna((df['FG3M'] / df['FG3A'].replace(0, 1)) * 100)
                elif col == 'FT_PCT' and 'FTM' in df.columns and 'FTA' in df.columns:
                    df[col] = df[col].fillna((df['FTM'] / df['FTA'].replace(0, 1)) * 100)
                else:
                    df[col] = df[col].fillna(0)
            else:
                df[col] = df[col].fillna(0)
    
    if 'PLUS_MINUS' in df.columns:
        df['PLUS_MINUS'] = df['PLUS_MINUS'].fillna(0)
    
    return df

games_details = clean_games_details(games_details)


def convert_data_types(df, df_name):

    if df_name == 'games':
        df['GAME_DATE_EST'] = pd.to_datetime(df['GAME_DATE_EST'])
        df['SEASON'] = df['SEASON'].astype(int)
        df['HOME_TEAM_ID'] = df['HOME_TEAM_ID'].astype(int)
        df['VISITOR_TEAM_ID'] = df['VISITOR_TEAM_ID'].astype(int)
        df['PTS_home'] = pd.to_numeric(df['PTS_home'], errors='coerce')
        df['PTS_away'] = pd.to_numeric(df['PTS_away'], errors='coerce')
        df['HOME_TEAM_WINS'] = df['HOME_TEAM_WINS'].astype(int)
        
    elif df_name == 'games_details':
        # Convert minutes to numeric (handle MM:SS format)
        def minutes_to_numeric(time_str):
            try:
                if pd.isna(time_str) or time_str == '' or time_str == '0:00':
                    return 0
                if ':' in str(time_str):
                    minutes, seconds = map(int, str(time_str).split(':'))
                    return minutes + seconds/60
                return float(time_str)
            except:
                return 0
        
        df['MIN'] = df['MIN'].apply(minutes_to_numeric)
        
        
        numeric_cols = ['FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 
                       'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS']
        for col in numeric_cols:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
        
        
        pct_cols = ['FG_PCT', 'FG3_PCT', 'FT_PCT']
        for col in pct_cols:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')
    
    elif df_name == 'players':
        df['PLAYER_ID'] = df['PLAYER_ID'].astype(int)

    
    elif df_name == 'teams':
        df['TEAM_ID'] = df['TEAM_ID'].astype(int)
        df['YEARFOUNDED'] = pd.to_numeric(df['YEARFOUNDED'], errors='coerce')
    
    return df

games = convert_data_types(games, 'games')
games_details = convert_data_types(games_details, 'games_details')
players = convert_data_types(players, 'players')
teams = convert_data_types(teams, 'teams')


def remove_duplicates_smart(df, df_name):

    if df_name == 'games':
       
        df = df.drop_duplicates(subset=['GAME_ID'], keep='first')
    
    elif df_name == 'games_details':
       
        df = df.drop_duplicates(subset=['GAME_ID', 'PLAYER_ID'], keep='first')
    
    elif df_name == 'players':
      
        df = df.drop_duplicates(subset=['PLAYER_ID'], keep='first')
    
    elif df_name == 'teams':
      
        df = df.drop_duplicates(subset=['TEAM_ID'], keep='first')
    
    elif df_name == 'ranking':
       
        df['STANDINGSDATE'] = pd.to_datetime(df['STANDINGSDATE'])
        df = df.drop_duplicates(subset=['TEAM_ID', 'STANDINGSDATE'], keep='first')
    
    return df

games = remove_duplicates_smart(games, 'games')
games_details = remove_duplicates_smart(games_details, 'games_details')
players = remove_duplicates_smart(players, 'players')
teams = remove_duplicates_smart(teams, 'teams')
ranking = remove_duplicates_smart(ranking, 'ranking')


def data_quality_checks(games, games_details):
    print("\n=== Data Quality Checks ===")
    

    print("\n1. Invalid values check:")
    
    invalid_points = games_details[games_details['PTS'] < 0]
    print(f"Rows with negative points: {len(invalid_points)}")
    
    invalid_minutes = games_details[(games_details['MIN'] < 0) | (games_details['MIN'] > 70)]
    print(f"Rows with invalid minutes: {len(invalid_minutes)}")
    
    invalid_fg = games_details[games_details['FGA'] < games_details['FGM']]
    print(f"Rows where FGA < FGM: {len(invalid_fg)}")
    
    print("\n2. Outlier analysis:")
    outlier_cols = ['PTS', 'REB', 'AST', 'MIN']
    
    for col in outlier_cols:
        if col in games_details.columns:
            q1 = games_details[col].quantile(0.01)
            q3 = games_details[col].quantile(0.99)
            outliers = games_details[(games_details[col] < q1) | (games_details[col] > q3)]
            print(f"{col} outliers (beyond 1st and 99th percentile): {len(outliers)}")
    
    print("\n3. Game consistency check:")
    
    games_with_details = games.merge(
        games_details.groupby('GAME_ID')['PTS'].sum().reset_index(),
        on='GAME_ID',
        how='inner'
    )
    
    print(f"Games in both datasets: {len(games_with_details)}")
    
    print("\n4. Season consistency check:")
    print(f"Seasons in games: {sorted(games['SEASON'].unique())}")
    
    return games_details

games_details = data_quality_checks(games, games_details)

def fix_invalid_values(df):

   
    df.loc[df['MIN'] > 70, 'MIN'] = 48  
    df.loc[df['MIN'] < 0, 'MIN'] = 0
    
   
    df.loc[df['FGA'] < df['FGM'], 'FGA'] = df['FGM']
    df.loc[df['FG3A'] < df['FG3M'], 'FG3A'] = df['FG3M']
    df.loc[df['FTA'] < df['FTM'], 'FTA'] = df['FTM']
    
 
    df['FG_PCT'] = np.where(df['FGA'] > 0, (df['FGM'] / df['FGA']), 0)
    df['FG3_PCT'] = np.where(df['FG3A'] > 0, (df['FG3M'] / df['FG3A']), 0)
    df['FT_PCT'] = np.where(df['FTA'] > 0, (df['FTM'] / df['FTA']), 0)
    
    return df

games_details = fix_invalid_values(games_details)

print(f"\nFinal dataset shapes:")
print(f"Games: {games.shape}")
print(f"Games Details: {games_details.shape}")
print(f"Players: {players.shape}")
print(f"Teams: {teams.shape}")
print(f"Ranking: {ranking.shape}")


=== Data Quality Checks ===

1. Invalid values check:
Rows with negative points: 0
Rows with invalid minutes: 21
Rows where FGA < FGM: 0

2. Outlier analysis:
PTS outliers (beyond 1st and 99th percentile): 4836
REB outliers (beyond 1st and 99th percentile): 5084
AST outliers (beyond 1st and 99th percentile): 4312
MIN outliers (beyond 1st and 99th percentile): 11068

3. Game consistency check:
Games in both datasets: 26523

4. Season consistency check:
Seasons in games: [np.int64(2003), np.int64(2004), np.int64(2005), np.int64(2006), np.int64(2007), np.int64(2008), np.int64(2009), np.int64(2010), np.int64(2011), np.int64(2012), np.int64(2013), np.int64(2014), np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022)]

Final dataset shapes:
Games: (26622, 21)
Games Details: (558490, 29)
Players: (1769, 4)
Teams: (30, 14)
Ranking: (210313, 13)


In [11]:
games_clean = games.dropna(subset=['PTS_home', 'PTS_away'])

pct_columns = ['FG_PCT_home', 'FT_PCT_home', 'FG3_PCT_home', 
               'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away']

for col in pct_columns:
    if col in games.columns:
        games[col] = games[col].fillna(games[col].median())

games_details_clean = games_details.drop('COMMENT', axis=1)

games_details_clean['NICKNAME'] = games_details_clean['NICKNAME'].fillna('')

# Handle START_POSITION - create a binary feature
games_details_clean['IS_STARTER'] = games_details_clean['START_POSITION'].notna().astype(int)
games_details_clean['START_POSITION'] = games_details_clean['START_POSITION'].fillna('BENCH')

ranking_clean = ranking.drop('RETURNTOPLAY', axis=1)

print("Missing values after cleaning:")
print(f"Games: {games_clean.isnull().sum().sum()} total missing values")
print(f"Games details: {games_details_clean.isnull().sum().sum()} total missing values")
print(f"Ranking: {ranking_clean.isnull().sum().sum()} total missing values")



Missing values after cleaning:
Games: 0 total missing values
Games details: 0 total missing values
Ranking: 0 total missing values


In [12]:
games_clean.to_csv("games_clean.csv", index=False)
games_details_clean.to_csv("games_details_clean.csv", index=False)
ranking_clean.to_csv("ranking_clean.csv", index=False)
players.to_csv("players_clean.csv", index=False)
teams.to_csv("teams_clean.csv", index=False)

print("CSV files have been saved.")

CSV files have been saved.
